# Determinants

There's a `det()` function within the module `LinearAlgebra`.
Let's compare the speeds of this determinant function with respect to the identity using the cross product `×()` and dot product `⋅()` operators within the same module.

We will exploit the following identity.
```
det([a,b,c]) = a × b ⋅ c
```
and in comparing their `@benchmark`s.

In [1]:
using Pkg;
Pkg.activate(".")
Pkg.add("LinearAlgebra")
Pkg.add("BenchmarkTools")
Pkg.update()
Pkg.status()

  Activating environment at `~/Documents/GitHub/hello-julia/src/Matrices/Project.toml`
    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/hello-julia/src/Matrices/Project.toml`
  No Changes to `~/Documents/GitHub/hello-julia/src/Matrices/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/hello-julia/src/Matrices/Project.toml`
  No Changes to `~/Documents/GitHub/hello-julia/src/Matrices/Manifest.toml`
    Updating registry at `~/.julia/registries/General`
  No Changes to `~/Documents/GitHub/hello-julia/src/Matrices/Project.toml`
  No Changes to `~/Documents/GitHub/hello-julia/src/Matrices/Manifest.toml`


      Status `~/Documents/GitHub/hello-julia/src/Matrices/Project.toml`
  [6e4b80f9] BenchmarkTools v1.2.0
  [37e2e46d] LinearAlgebra


In [2]:
using LinearAlgebra
using BenchmarkTools

In [3]:
a = rand(3);
b = rand(3);
c = rand(3);

a′ = a;

A = [a b c];

println("$(a==a′) : a′ is equal to a.")
println("$(a===a′) : a′ is $((a===a′) ? "also " : "")identical to a.")

println("a[2]=$(a[2]), a′[2]=$(a′[2]).")
println("  setting new random values for a′[2]...")
a′[2] = rand(); #new value
println("a[2]=$(a[2]), a′[2]=$(a′[2]).")

true : a′ is equal to a.
true : a′ is also identical to a.
a[2]=0.11435700389430381, a′[2]=0.11435700389430381.
  setting new random values for a′[2]...
a[2]=0.9748342965033607, a′[2]=0.9748342965033607.


In [4]:
A = [a b c];
println("$(A[:,1] == a) : A[:,1] is equal to a.")
println("$(A[:,1] === a) : A[:,1] is $((A[:,1] === a) ? "also " : "")identical to a.")

true : A[:,1] is equal to a.
false : A[:,1] is identical to a.


In [5]:
triple_det(A::Matrix) = (A[:,1] × A[:,2]) ⋅ A[:,3]

triple_det (generic function with 1 method)

In [6]:
bmark0 = @benchmark for _ in 1:1_000_000 det($A) end

BenchmarkTools.Trial: 20 samples with 1 evaluation.
 Range (min … max):  252.537 ms … 282.159 ms  ┊ GC (min … max): 2.22% … 2.08%
 Time  (median):     263.287 ms               ┊ GC (median):    2.18%
 Time  (mean ± σ):   263.983 ms ±   6.906 ms  ┊ GC (mean ± σ):  2.14% ± 0.15%

  ▁▁      ▁  ▁  ▁▁   ▁█▁ █  ▁▁  ▁   ▁▁▁   ▁                   ▁  
  ██▁▁▁▁▁▁█▁▁█▁▁██▁▁▁███▁█▁▁██▁▁█▁▁▁███▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  253 ms           Histogram: frequency by time          282 ms <

 Memory estimate: 259.40 MiB, allocs estimate: 2000000.

In [7]:
bmark1 = @benchmark for _ in 1:1_000_000 triple_det($A) end

BenchmarkTools.Trial: 25 samples with 1 evaluation.
 Range (min … max):  185.481 ms … 276.567 ms  ┊ GC (min … max): 5.88% … 4.50%
 Time  (median):     192.532 ms               ┊ GC (median):    5.68%
 Time  (mean ± σ):   200.571 ms ±  23.631 ms  ┊ GC (mean ± σ):  5.42% ± 0.41%

  ▄▁█▁█    ▄                                                     
  █████▁▆▆▁█▁▁▁▆▆▁▁▁▁▁▁▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▁▆ ▁
  185 ms           Histogram: frequency by time          277 ms <

 Memory estimate: 427.25 MiB, allocs estimate: 4000000.

In [8]:
mratio = bmark1.memory/bmark0.memory;
println("Memory ratio:");
println("  mem[my_det]/mem[det] = $(round(mratio,digits=5))");
println("  mem[det]/mem[my_det] = $(round(1/mratio,digits=5)).");

Memory ratio:
  mem[my_det]/mem[det] = 1.64706
  mem[det]/mem[my_det] = 0.60714.


In [9]:
tratio = median(bmark1.times)/median(bmark0.times);
println("Time speed ratio:")
println("  time[my_det]/time[det]: $(round(tratio,digits=5))")
println("  time[det]/time[my_det]: $(round(1/tratio,digits=5))")

Time speed ratio:
  time[my_det]/time[det]: 0.73126
  time[det]/time[my_det]: 1.3675


In [10]:
mt1 = (bmark1.memory)*median(bmark1.times)

8.6254399168e16

In [11]:
mt0 = (bmark0.memory)*median(bmark0.times)

7.1614092016e16

In [12]:
Δmt = (mt1 - mt0)/(mt1 + mt0) * 100.0;
println("mt deviation of bench1 from bench0: $(round(Δmt,digits=2)) %.")
rmt = mt1/mt0;
println("Ratio: mt1/mt0 = $(round(rmt,digits=5)).")

mt deviation of bench1 from bench0: 9.27 %.
Ratio: mt1/mt0 = 1.20443.
